In [23]:
import pandas as pd

In [3]:
df1 = pd.read_csv('sismos_japon-1960-2000.csv')
df2 = pd.read_csv('sismos_japon-2000-2010.csv')
df3 = pd.read_csv('sismos_japon-2010-2018.csv')
df4 = pd.read_csv('sismos_japon-2018-2023.csv')

df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [4]:
df = df.rename(columns={'地震の発生日': 'Fecha',
                        '地震の発生時刻': 'Hora',
                        '震央地名': 'Epicentro',
                        '緯度': 'Latitud',
                        '経度': 'Longitud',
                        '深さ': 'Profundidad',
                        '最大震度': 'Máxima intensidad sísmica',
                        'Ｍ':'Mag'})


In [5]:
df.head()

,Fecha,Hora,Epicentro,Latitud,Longitud,Profundidad,Mag,Máxima intensidad sísmica
0,1999/12/16,22:47:09.6,栃木県北部,36°38.4′N,139°25.8′E,9 km,4.2,震度４
1,1999/11/29,21:34:04.1,愛知県西部,35°06.7′N,137°01.5′E,45 km,4.8,震度４
2,1999/11/10,03:19:17.1,熊本県熊本地方,32°43.6′N,130°47.8′E,13 km,4.2,震度４
3,1999/11/07,03:34:01.6,福井県沖,36°03.5′N,135°47.6′E,15 km,5.0,震度４
4,1999/10/30,06:25:50.8,瀬戸内海中部,34°02.8′N,133°30.1′E,13 km,4.6,震度４


In [6]:

df['Máxima intensidad sísmica'] = df['Máxima intensidad sísmica'].replace({'震度７': 'Intensidad sísmica 7',
                                        '震度３': 'Intensidad sísmica 3',
                                        '震度６': 'Intensidad sísmica 6',
                                        '震度５強': 'Intensidad sísmica 5 fuerte',
                                        '震度４': 'Intensidad sísmica 4',
                                        '震度６弱': 'Intensidad sísmica 6 débil',
                                        '震度２': 'Intensidad sísmica 2',
                                        '震度５': 'Intensidad sísmica 5',
                                        '震度１': 'Intensidad sísmica 1',
                                        '震度６強': 'Intensidad sísmica 6 fuerte',
                                        '震度５弱': 'Intensidad sísmica 5 débil',})


In [7]:
df.head()

,Fecha,Hora,Epicentro,Latitud,Longitud,Profundidad,Mag,Máxima intensidad sísmica
0,1999/12/16,22:47:09.6,栃木県北部,36°38.4′N,139°25.8′E,9 km,4.2,Intensidad sísmica 4
1,1999/11/29,21:34:04.1,愛知県西部,35°06.7′N,137°01.5′E,45 km,4.8,Intensidad sísmica 4
2,1999/11/10,03:19:17.1,熊本県熊本地方,32°43.6′N,130°47.8′E,13 km,4.2,Intensidad sísmica 4
3,1999/11/07,03:34:01.6,福井県沖,36°03.5′N,135°47.6′E,15 km,5.0,Intensidad sísmica 4
4,1999/10/30,06:25:50.8,瀬戸内海中部,34°02.8′N,133°30.1′E,13 km,4.6,Intensidad sísmica 4


In [49]:
import requests, uuid, json

# Add your key and endpoint
key = "d86679bda1ad49a8b0a8b5f36994b979"
endpoint = "https://api.cognitive.microsofttranslator.com"

# location, also known as region.
# required if you're using a multi-service or regional (not global) resource. It can be found in the Azure portal on the Keys and Endpoint page.
location = "eastus"

path = '/translate'
constructed_url = endpoint + path

params = {
    'api-version': '3.0',
    'from': 'ja',
    'to': ['es']
}

headers = {
    'Ocp-Apim-Subscription-Key': key,
    # location required if you're using a multi-service or regional (not global) resource.
    'Ocp-Apim-Subscription-Region': location,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

# You can pass more than one object in body.
body = [{
    'text': '千島列島東方'
}]

In [50]:
response = requests.post(constructed_url, params=params, headers=headers, json=body).json()[0]['translations'][0]['text']

In [10]:
traducciones = []
for text in df['Epicentro']:
    body = [{
        'text': text
    }]
    response = requests.post(constructed_url, params=params, headers=headers, json=body).json()[0]['translations'][0]['text']
    traducciones.append(response)

In [11]:
df['Epicentro'] = traducciones

In [12]:
def int_lat_long(coord):
    int_coord, min_sec = coord.split('°')
    mins, secs = min_sec.split('.')
    secs = secs.replace('′N', '')
    secs = int(secs.replace('′E', ''))/600

    int_coord = int(int_coord)
    mins = int(mins)/60

    return int_coord + mins + secs

In [13]:
df['Latitud'] = df['Latitud'].apply(int_lat_long).astype(float)
df['Longitud'] = df['Longitud'].apply(int_lat_long).astype(float)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2851 entries, 0 to 2850
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Fecha                      2851 non-null   object 
 1   Hora                       2851 non-null   object 
 2   Epicentro                  2851 non-null   object 
 3   Latitud                    2851 non-null   float64
 4   Longitud                   2851 non-null   float64
 5   Profundidad                2851 non-null   object 
 6   Mag                        2851 non-null   object 
 7   Máxima intensidad sísmica  2851 non-null   object 
dtypes: float64(2), object(6)
memory usage: 178.3+ KB


In [15]:
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%Y/%m/%d')

In [16]:
df['Hora'] = pd.to_datetime(df['Hora'], format='%H:%M:%S.%f', errors='coerce')
df['Hora'] = df['Hora'].dt.strftime('%H:%M:%S')

In [17]:
df['Mag'] = pd.to_numeric(df['Mag'], errors='coerce')

In [18]:
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
df

,Fecha,Hora,Epicentro,Latitud,Longitud,Profundidad,Mag,Máxima intensidad sísmica
0,1999-12-16,22:47:09,Norte de Tochigi,36.640000,139.430000,9 km,4.2,Intensidad sísmica 4
1,1999-11-29,21:34:04,Prefectura Occidental de Aichi,35.111667,137.025000,45 km,4.8,Intensidad sísmica 4
2,1999-11-10,03:19:17,"Región de Kumamoto, Kumamoto",32.726667,130.796667,13 km,4.2,Intensidad sísmica 4
3,1999-11-07,03:34:01,Frente a la costa de Fukui,36.058333,135.793333,15 km,5.0,Intensidad sísmica 4
4,1999-10-30,06:25:50,Mar Interior de Seto Central,34.046667,133.501667,13 km,4.6,Intensidad sísmica 4
...,...,...,...,...,...,...,...,...
2846,2018-02-19,03:31:36,Bungo Waterworks,32.903333,132.161667,40 km,5.0,Intensidad sísmica 4
2847,2018-02-13,14:39:44,Prefectura de Nagano del Sur,35.865000,137.593333,6 km,4.1,Intensidad sísmica 4
2848,2018-01-24,19:51:19,Frente a la costa oriental de la prefectura de...,41.008333,142.446667,34 km,6.3,Intensidad sísmica 4
2849,2018-01-14,18:07:33,Fuera de Urakawa,41.936667,143.046667,41 km,4.7,Intensidad sísmica 4


Guardando traducciones para el avance

In [8]:
df.to_csv('../data/sismos_japon_completo.csv', index=False)